In [ ]:
import pandas as pd
import pprint
import math
import os

def safe_tuple(val_min, val_max):
    """
    Convert two numeric (or empty/NaN) cells into a (min_val, max_val) tuple.
    If they're blank, we return (None, None).
    """
    # If either cell is NaN, treat it as None
    if isinstance(val_min, float) and math.isnan(val_min):
        val_min = None
    if isinstance(val_max, float) and math.isnan(val_max):
        val_max = None
    return (val_min, val_max)

def build_dhw_lookup_dict_from_excel(excel_path):
    """
    Reads the given Excel file with columns like:
      calibration_stage, dhw_key,
      occupant_density_m2_per_person_min, occupant_density_m2_per_person_max,
      liters_per_person_per_day_min, liters_per_person_per_day_max,
      default_tank_volume_liters_min, default_tank_volume_liters_max,
      default_heater_capacity_w_min, default_heater_capacity_w_max,
      setpoint_c_min, setpoint_c_max,
      usage_split_factor_min, usage_split_factor_max,
      peak_hours_min, peak_hours_max,
      sched_morning_min, sched_morning_max,
      sched_peak_min, sched_peak_max,
      sched_afternoon_min, sched_afternoon_max,
      sched_evening_min, sched_evening_max

    Returns a dictionary structured like:
      dhw_lookup = {
        "TABLE_13_1_KWH_PER_M2": {...},  # We'll hard-code or skip
        "pre_calibration": {
           "Corner House": {
              "occupant_density_m2_per_person_range": (..),
              "liters_per_person_per_day_range": (..),
              ...
           },
           ...
        },
        "post_calibration": {
           ...
        }
      }
    """
    df = pd.read_excel(excel_path)

    # Initialize the main dict
    dhw_lookup = {}

    # Optionally define or skip the TABLE_13_1_KWH_PER_M2 here:
    dhw_lookup["TABLE_13_1_KWH_PER_M2"] = {
        "Meeting Function":       2.8,
        "Office Function":        1.4,
        "Retail Function":        1.4,
        "Healthcare Function":    15.3,
        "Education Function":     1.4,
        "Sport Function":         12.5,
        "Cell Function":          4.2,
        "Industrial Function":    1.2,
        "Accommodation Function": 12.5,
        "Other Use Function":     2.4
    }

    # Build up the "pre_calibration"/"post_calibration" entries
    for _, row in df.iterrows():
        stage = str(row["calibration_stage"]).strip()
        dhw_key = str(row["dhw_key"]).strip()

        # Ensure stage dict exists
        if stage not in dhw_lookup:
            dhw_lookup[stage] = {}

        # Ensure dhw_key dict exists under this stage
        if dhw_key not in dhw_lookup[stage]:
            dhw_lookup[stage][dhw_key] = {}

        # occupant_density
        dhw_lookup[stage][dhw_key]["occupant_density_m2_per_person_range"] = \
            safe_tuple(row["occupant_density_m2_per_person_min"],
                       row["occupant_density_m2_per_person_max"])

        # liters_per_person_per_day
        dhw_lookup[stage][dhw_key]["liters_per_person_per_day_range"] = \
            safe_tuple(row["liters_per_person_per_day_min"],
                       row["liters_per_person_per_day_max"])

        # default_tank_volume_liters
        dhw_lookup[stage][dhw_key]["default_tank_volume_liters_range"] = \
            safe_tuple(row["default_tank_volume_liters_min"],
                       row["default_tank_volume_liters_max"])

        # default_heater_capacity_w
        dhw_lookup[stage][dhw_key]["default_heater_capacity_w_range"] = \
            safe_tuple(row["default_heater_capacity_w_min"],
                       row["default_heater_capacity_w_max"])

        # setpoint_c
        dhw_lookup[stage][dhw_key]["setpoint_c_range"] = \
            safe_tuple(row["setpoint_c_min"], row["setpoint_c_max"])

        # usage_split_factor
        dhw_lookup[stage][dhw_key]["usage_split_factor_range"] = \
            safe_tuple(row["usage_split_factor_min"],
                       row["usage_split_factor_max"])

        # peak_hours
        dhw_lookup[stage][dhw_key]["peak_hours_range"] = \
            safe_tuple(row["peak_hours_min"], row["peak_hours_max"])

        # sched_morning
        dhw_lookup[stage][dhw_key]["sched_morning_range"] = \
            safe_tuple(row["sched_morning_min"],
                       row["sched_morning_max"])

        # sched_peak
        dhw_lookup[stage][dhw_key]["sched_peak_range"] = \
            safe_tuple(row["sched_peak_min"],
                       row["sched_peak_max"])

        # sched_afternoon
        dhw_lookup[stage][dhw_key]["sched_afternoon_range"] = \
            safe_tuple(row["sched_afternoon_min"],
                       row["sched_afternoon_max"])

        # sched_evening
        dhw_lookup[stage][dhw_key]["sched_evening_range"] = \
            safe_tuple(row["sched_evening_min"],
                       row["sched_evening_max"])

    return dhw_lookup

def main():
    # Path to your Excel file
    excel_file = r"D:\Documents\E_Plus_2030_py\lookup_pys\dhw_lookup.xlsx"

    # Build the dictionary
    dhw_dict = build_dhw_lookup_dict_from_excel(excel_file)

    # Construct the path where we want to write the new "dhw_lookup.py"
    out_path = os.path.join(
        os.path.dirname(excel_file),  # same folder
        "dhw_lookup.py"               # output filename
    )

    # Write the dictionary as a .py file
    with open(out_path, "w", encoding="utf-8") as f:
        f.write("# Auto-generated from Excel\n\n")
        f.write("dhw_lookup = ")
        f.write(pprint.pformat(dhw_dict, sort_dicts=False))
        f.write("\n")

    print(f"Generated dhw_lookup.py at: {out_path}")

if __name__ == "__main__":
    main()


Generated dhw_lookup.py at: D:\Documents\E_Plus_2030_py\lookup_pys\dhw_lookup.py


In [9]:
import pandas as pd
import pprint
import ast
import os

def parse_tuple_string(s):
    """
    Given a string like '(0.0, 0.0)', '(2100, 2300)',
    parse it into a real Python tuple of floats or ints.

    We'll use ast.literal_eval to safely parse the string.
    If the string is empty or invalid, return None or (None, None).
    """
    if not isinstance(s, str) or not s.strip():
        return (None, None)
    try:
        # e.g. s = "(0.0, 0.0)" => ast.literal_eval -> (0.0, 0.0)
        val = ast.literal_eval(s.strip())
        # Ensure it's a tuple
        if isinstance(val, tuple) and len(val) == 2:
            return val
        else:
            # If it's not a 2-element tuple, just return as-is or (None, None)
            return (None, None)
    except:
        return (None, None)

def main():
    # 1) Path to your Excel file
    excel_file = r"D:\Documents\E_Plus_2030_py\lookup_pys\lighting_lookup.xlsx"

    # 2) Read the Excel into a pandas DataFrame
    df = pd.read_excel(excel_file)

    # 3) Prepare the final dictionary
    lighting_lookup = {}

    # 4) Iterate each row to build the nested dict
    for _, row in df.iterrows():
        stage = str(row["Calibration Stage"]).strip()        # e.g. "Pre-calibration"
        category = str(row["Category"]).strip()              # e.g. "Residential" or "Non-Residential"
        subtype = str(row["Sub-Type/Function"]).strip()      # e.g. "Corner House", "Meeting Function"

        # Convert the stage to lowercase with underscores if desired
        # e.g. "Pre-calibration" => "pre_calibration"
        # or you can preserve as-is. Let's do the typical approach:
        stage_key = stage.lower().replace("-", "_")

        # Ensure top-level key exists
        if stage_key not in lighting_lookup:
            lighting_lookup[stage_key] = {}

        # Ensure category key exists
        if category not in lighting_lookup[stage_key]:
            lighting_lookup[stage_key][category] = {}

        # Now parse each parameter column
        lights_wm2        = parse_tuple_string(row["LIGHTS_WM2_range"])
        parasitic_wm2     = parse_tuple_string(row["PARASITIC_WM2_range"])
        td_range          = parse_tuple_string(row["tD_range"])
        tn_range          = parse_tuple_string(row["tN_range"])
        frac_rad_lights   = parse_tuple_string(row["lights_fraction_radiant_range"])
        frac_vis_lights   = parse_tuple_string(row["lights_fraction_visible_range"])
        frac_repl_lights  = parse_tuple_string(row["lights_fraction_replaceable_range"])
        frac_rad_equip    = parse_tuple_string(row["equip_fraction_radiant_range"])
        frac_lost_equip   = parse_tuple_string(row["equip_fraction_lost_range"])

        # Insert into the dictionary
        lighting_lookup[stage_key][category][subtype] = {
            "LIGHTS_WM2_range": lights_wm2,
            "PARASITIC_WM2_range": parasitic_wm2,
            "tD_range": td_range,
            "tN_range": tn_range,
            "lights_fraction_radiant_range": frac_rad_lights,
            "lights_fraction_visible_range": frac_vis_lights,
            "lights_fraction_replaceable_range": frac_repl_lights,
            "equip_fraction_radiant_range": frac_rad_equip,
            "equip_fraction_lost_range": frac_lost_equip,
        }

    # 5) Write out to a .py file (same folder as the Excel)
    out_path = os.path.join(
        os.path.dirname(excel_file),
        "lighting_lookup.py"
    )

    with open(out_path, "w", encoding="utf-8") as f:
        # Possibly add a docstring or header
        f.write("# Auto-generated from lighting_lookup.xlsx\n\n")
        f.write("lighting_lookup = ")
        f.write(pprint.pformat(lighting_lookup, sort_dicts=False))
        f.write("\n")

    print(f"Created: {out_path}")

if __name__ == "__main__":
    main()


Created: D:\Documents\E_Plus_2030_py\lookup_pys\lighting_lookup.py


In [10]:
import pandas as pd
import ast
import os
import pprint

def parse_tuple_string(s):
    """
    Convert a string like '(20.0, 21.0)' into a real tuple (20.0, 21.0).
    If s is empty or invalid, return (None, None).
    """
    if not s or not isinstance(s, str) or not s.strip():
        return (None, None)
    try:
        return ast.literal_eval(s.strip())  # safely parse to tuple
    except:
        return (None, None)

def parse_schedule_details(s):
    """
    Convert a string like:
      'day_start=07:00; day_end=23:00; occ: weekday=FullOccupancy, weekend=FullOccupancy'
    or
      'wd_start=08:00; wd_end=22:00; we_start=08:00; we_end=18:00; occ: wd=MeetingFunction_Weekday, we=MeetingFunction_Weekend'
    into a dict like:
      {
        "day_start": "07:00",
        "day_end": "23:00",
        "occupancy_schedule": { "weekday": "FullOccupancy", "weekend": "FullOccupancy" }
      }
    or for the wd_/we_ patterns:
      {
        "weekday_day_start": "08:00",
        "weekday_day_end":   "22:00",
        "weekend_day_start": "08:00",
        "weekend_day_end":   "18:00",
        "occupancy_schedule": {
          "weekday": "MeetingFunction_Weekday",
          "weekend": "MeetingFunction_Weekend"
        }
      }
    """
    details = {}
    occupancy = {}
    # Example tokens: ["day_start=07:00", "day_end=23:00", "occ: weekday=FullOccupancy, weekend=FullOccupancy"]
    # or ["wd_start=08:00", "wd_end=22:00", "we_start=08:00", "we_end=18:00", "occ: wd=MeetingFunction_Weekday, we=MeetingFunction_Weekend"]
    parts = s.split(";")
    for part in parts:
        part = part.strip()
        # e.g. "day_start=07:00" or "occ: weekday=FullOccupancy, weekend=FullOccupancy"
        if not part:
            continue

        if part.startswith("occ:"):
            # parse occupancy references => "occ: wd=..., we=..."
            # remove "occ:"
            occ_str = part[4:].strip()  # e.g. "weekday=FullOccupancy, weekend=FullOccupancy"
            # split by comma to get each
            occ_items = occ_str.split(",")
            for oitem in occ_items:
                oitem = oitem.strip()
                # e.g. "weekday=FullOccupancy" or "wd=MeetingFunction_Weekday"
                if "=" in oitem:
                    key, val = oitem.split("=", 1)
                    key = key.strip()
                    val = val.strip()
                    # unify "weekday"/"wd" => "weekday", "weekend"/"we" => "weekend"
                    if key in ["wd", "weekday"]:
                        occupancy["weekday"] = val
                    elif key in ["we", "weekend"]:
                        occupancy["weekend"] = val
        else:
            # parse key=val
            if "=" in part:
                key, val = part.split("=", 1)
                key = key.strip()
                val = val.strip()
                # check if it's day_start / wd_start etc.
                # We'll unify them to the hvac_lookup.py style:
                if key == "day_start":
                    details["day_start"] = val
                elif key == "day_end":
                    details["day_end"] = val
                elif key == "wd_start":
                    details["weekday_day_start"] = val
                elif key == "wd_end":
                    details["weekday_day_end"] = val
                elif key == "we_start":
                    details["weekend_day_start"] = val
                elif key == "we_end":
                    details["weekend_day_end"] = val
                else:
                    # fallback: store directly if you want
                    # e.g. details["something"] = val
                    details[key] = val

    # attach occupancy sub-dict if present
    if occupancy:
        details["occupancy_schedule"] = occupancy

    return details

def main():
    excel_file = r"D:\Documents\E_Plus_2030_py\lookup_pys\hvac_lookup.xlsx"
    df = pd.read_excel(excel_file)

    # final structure
    hvac_lookup = {}

    for _, row in df.iterrows():
        # 1) Read columns
        calib_stage = str(row["Calib Stage"]).strip()       # e.g. "pre_calibration"
        scenario    = str(row["Scenario"]).strip()          # e.g. "scenario1"
        build_func  = str(row["Build Func"]).strip()        # e.g. "residential" or "non_residential"
        subtype     = str(row["Subtype"]).strip()           # e.g. "Corner House", "Meeting Function"
        age_range   = str(row["Age Range"]).strip()         # e.g. "2015 and later"

        # parse tuple columns
        hd_day   = parse_tuple_string(row["heating_day_setpoint_range"])
        hd_night = parse_tuple_string(row["heating_night_setpoint_range"])
        cd_day   = parse_tuple_string(row["cooling_day_setpoint_range"])
        cd_night = parse_tuple_string(row["cooling_night_setpoint_range"])
        max_ht   = parse_tuple_string(row["max_heating_supply_air_temp_range"])
        min_cl   = parse_tuple_string(row["min_cooling_supply_air_temp_range"])

        # schedule details
        sched_str = row["schedule_details"] if isinstance(row["schedule_details"], str) else ""
        sched_dict = parse_schedule_details(sched_str)

        # 2) Ensure the nested keys exist
        if calib_stage not in hvac_lookup:
            hvac_lookup[calib_stage] = {}
        if scenario not in hvac_lookup[calib_stage]:
            hvac_lookup[calib_stage][scenario] = {}
        if build_func not in hvac_lookup[calib_stage][scenario]:
            hvac_lookup[calib_stage][scenario][build_func] = {}
        if subtype not in hvac_lookup[calib_stage][scenario][build_func]:
            hvac_lookup[calib_stage][scenario][build_func][subtype] = {}

        # 3) Insert the final dictionary under hvac_lookup[calib_stage][scenario][build_func][subtype][age_range]
        hvac_lookup[calib_stage][scenario][build_func][subtype][age_range] = {
            "heating_day_setpoint_range": hd_day,
            "heating_night_setpoint_range": hd_night,
            "cooling_day_setpoint_range": cd_day,
            "cooling_night_setpoint_range": cd_night,
            "max_heating_supply_air_temp_range": max_ht,
            "min_cooling_supply_air_temp_range": min_cl,
            "schedule_details": sched_dict
        }

    # write out hvac_lookup.py
    out_path = os.path.join(os.path.dirname(excel_file), "hvac_lookup.py")
    with open(out_path, "w", encoding="utf-8") as f:
        f.write("# Auto-generated from hvac_lookup.xlsx\n\n")
        f.write("hvac_lookup = ")
        f.write(pprint.pformat(hvac_lookup, sort_dicts=False))
        f.write("\n")

    print(f"Created hvac_lookup.py at: {out_path}")

if __name__ == "__main__":
    main()


Created hvac_lookup.py at: D:\Documents\E_Plus_2030_py\lookup_pys\hvac_lookup.py


In [11]:
import pandas as pd
import ast
import os
import pprint
import math

def parse_tuple_or_none(val):
    """
    If 'val' is a string like '(0.12, 0.15)', parse into a tuple (0.12, 0.15).
    If empty or invalid, return None.
    """
    if not val or not isinstance(val, str) or not val.strip():
        return None
    try:
        return ast.literal_eval(val.strip())  # safely parse the string
    except:
        return None

def parse_float_or_none(val):
    """
    Convert the cell to float if possible. If blank or NaN, return None.
    """
    if val is None or (isinstance(val, float) and math.isnan(val)):
        return None
    try:
        return float(val)
    except:
        return None

def main():
    # Path to your materials Excel file
    excel_file = r"D:\Documents\E_Plus_2030_py\lookup_pys\materials.xlsx"

    # Read the Excel
    df = pd.read_excel(excel_file)

    # We'll build this final dictionary:
    material_lookup = {}

    # Iterate rows
    for _, row in df.iterrows():
        # "Key" is the main dictionary key
        key = str(row["Key"]).strip()
        if not key:  # skip blank "Key" rows
            continue

        # Start building the sub-dict
        mat_dict = {}

        # For each column, if the cell is not blank, store it
        # in a manner matching your original materials_lookup.py.

        # 1) Straight string columns
        if pd.notna(row.get("Obj Type", None)):
            mat_dict["obj_type"] = str(row["Obj Type"]).strip()
        if pd.notna(row.get("Name", None)):
            mat_dict["Name"] = str(row["Name"]).strip()
        if pd.notna(row.get("Description", None)):
            desc = str(row["Description"]).strip()
            if desc:
                mat_dict["Description"] = desc
        if pd.notna(row.get("Roughness", None)):
            mat_dict["Roughness"] = str(row["Roughness"]).strip()
        if pd.notna(row.get("Optical_Data_Type", None)):
            mat_dict["Optical_Data_Type"] = str(row["Optical_Data_Type"]).strip()
        if pd.notna(row.get("Solar_Diffusing", None)):
            mat_dict["Solar_Diffusing"] = str(row["Solar_Diffusing"]).strip()

        # 2) Range columns (parse tuple if present)
        thickness_range = parse_tuple_or_none(row.get("Thickness_range", None))
        if thickness_range is not None:
            mat_dict["Thickness_range"] = thickness_range

        conductivity_range = parse_tuple_or_none(row.get("Conductivity_range", None))
        if conductivity_range is not None:
            mat_dict["Conductivity_range"] = conductivity_range

        density_range = parse_tuple_or_none(row.get("Density_range", None))
        if density_range is not None:
            mat_dict["Density_range"] = density_range

        specific_heat_range = parse_tuple_or_none(row.get("Specific_Heat_range", None))
        if specific_heat_range is not None:
            mat_dict["Specific_Heat_range"] = specific_heat_range

        thermal_abs_range = parse_tuple_or_none(row.get("Thermal_Absorptance_range", None))
        if thermal_abs_range is not None:
            mat_dict["Thermal_Absorptance_range"] = thermal_abs_range

        solar_abs_range = parse_tuple_or_none(row.get("Solar_Absorptance_range", None))
        if solar_abs_range is not None:
            mat_dict["Solar_Absorptance_range"] = solar_abs_range

        visible_abs_range = parse_tuple_or_none(row.get("Visible_Absorptance_range", None))
        if visible_abs_range is not None:
            mat_dict["Visible_Absorptance_range"] = visible_abs_range

        thermal_res_range = parse_tuple_or_none(row.get("Thermal_Resistance_range", None))
        if thermal_res_range is not None:
            mat_dict["Thermal_Resistance_range"] = thermal_res_range

        solar_trans_range = parse_tuple_or_none(row.get("Solar_Transmittance_range", None))
        if solar_trans_range is not None:
            mat_dict["Solar_Transmittance_range"] = solar_trans_range

        fsolrefl_range = parse_tuple_or_none(row.get("Front_Solar_Reflectance_range", None))
        if fsolrefl_range is not None:
            mat_dict["Front_Solar_Reflectance_range"] = fsolrefl_range

        bsolrefl_range = parse_tuple_or_none(row.get("Back_Solar_Reflectance_range", None))
        if bsolrefl_range is not None:
            mat_dict["Back_Solar_Reflectance_range"] = bsolrefl_range

        vis_trans_range = parse_tuple_or_none(row.get("Visible_Transmittance_range", None))
        if vis_trans_range is not None:
            mat_dict["Visible_Transmittance_range"] = vis_trans_range

        fvisrefl_range = parse_tuple_or_none(row.get("Front_Visible_Reflectance_range", None))
        if fvisrefl_range is not None:
            mat_dict["Front_Visible_Reflectance_range"] = fvisrefl_range

        bvisrefl_range = parse_tuple_or_none(row.get("Back_Visible_Reflectance_range", None))
        if bvisrefl_range is not None:
            mat_dict["Back_Visible_Reflectance_range"] = bvisrefl_range

        fir_emiss_range = parse_tuple_or_none(row.get("Front_IR_Emissivity_range", None))
        if fir_emiss_range is not None:
            mat_dict["Front_IR_Emissivity_range"] = fir_emiss_range

        bir_emiss_range = parse_tuple_or_none(row.get("Back_IR_Emissivity_range", None))
        if bir_emiss_range is not None:
            mat_dict["Back_IR_Emissivity_range"] = bir_emiss_range

        dirt_corr_range = parse_tuple_or_none(row.get("Dirt_Correction_Factor_range", None))
        if dirt_corr_range is not None:
            mat_dict["Dirt_Correction_Factor_range"] = dirt_corr_range

        # 3) Float columns (like IR_Transmittance)
        ir_trans = parse_float_or_none(row.get("IR_Transmittance", None))
        if ir_trans is not None:
            mat_dict["IR_Transmittance"] = ir_trans

        # 4) If "obj_type" was never set (blank in Excel), fallback to "MATERIAL"
        if "obj_type" not in mat_dict:
            mat_dict["obj_type"] = "MATERIAL"

        # 5) Insert into final dictionary
        material_lookup[key] = mat_dict

    # Write out materials_lookup.py
    out_path = os.path.join(os.path.dirname(excel_file), "materials_lookup.py")
    with open(out_path, "w", encoding="utf-8") as f:
        f.write("# Auto-generated from materials.xlsx\n\n")
        f.write('"""\nmaterials_lookup.py\n\n')
        f.write('Defines a dictionary `material_lookup` from your Excel table.\n"""\n\n')
        f.write("material_lookup = ")
        f.write(pprint.pformat(material_lookup, sort_dicts=False))
        f.write("\n")

    print(f"Created materials_lookup.py at: {out_path}")

if __name__ == "__main__":
    main()


Created materials_lookup.py at: D:\Documents\E_Plus_2030_py\lookup_pys\materials_lookup.py


In [1]:
import pandas as pd
import os
import pprint
import math

def float_or_none(val):
    """Convert val to float if possible, else None."""
    if val is None or (isinstance(val, float) and math.isnan(val)):
        return None
    try:
        return float(val)
    except:
        return None

def main():
    excel_path = r"D:\Documents\E_Plus_2030_py\lookup_pys\Vent_lookup.xlsx"
    df = pd.read_excel(excel_path)

    ventilation_lookup = {}

    # We'll group all rows by (scenario, calibration) because each block
    # builds a subdict with infiltration ranges, year_factor, system_control, etc.
    grouped = df.groupby(["Scenario", "Calibration"], dropna=False)

    for (scenario, calibration), group_df in grouped:
        scenario = str(scenario).strip()
        calibration_stage = str(calibration).strip()

        # Ensure top-level keys exist
        if scenario not in ventilation_lookup:
            ventilation_lookup[scenario] = {}
        if calibration_stage not in ventilation_lookup[scenario]:
            # Initialize the sub-structure
            ventilation_lookup[scenario][calibration_stage] = {
                "residential_infiltration_range": {},
                "non_res_infiltration_range": {},
                "year_factor_range": {},
                "system_control_range_res": {},
                "system_control_range_nonres": {},
                "fan_pressure_range": {},  # e.g. "res_mech", "nonres_intake", ...
                "hrv_sensible_eff_range": (None, None),
                "system_type_map": {
                    "residential": {},
                    "non_residential": {}
                }
            }

        subdict = ventilation_lookup[scenario][calibration_stage]
        # Quick references
        res_infil = subdict["residential_infiltration_range"]
        nonres_infil = subdict["non_res_infiltration_range"]
        year_factor = subdict["year_factor_range"]
        ctrl_res = subdict["system_control_range_res"]
        ctrl_nonres = subdict["system_control_range_nonres"]
        fanp = subdict["fan_pressure_range"]
        # HRV tuple: we'll store min & max as we go, or just pick last
        hrv_min, hrv_max = subdict["hrv_sensible_eff_range"]
        # system_type_map
        sys_map_res = subdict["system_type_map"]["residential"]
        sys_map_nonres = subdict["system_type_map"]["non_residential"]

        for _, row in group_df.iterrows():
            category = str(row["Category"]).strip()  # "residential" or "non_residential"
            year_range = str(row["Year Range"]).strip()  # e.g. "< 1945"
            bldg_type = str(row["Building/Function Type"]).strip()  # e.g. "Corner House", "Meeting Function"

            # infiltration
            inf_min = float_or_none(row["Inf Min"])
            inf_max = float_or_none(row["Inf Max"])

            # year factor
            yfac_min = float_or_none(row["Year Fac Min"])
            yfac_max = float_or_none(row["Year Fac Max"])

            # system letter
            sys_letter = str(row["System Ctrl Letter"]).strip() if not pd.isna(row["System Ctrl Letter"]) else ""

            # system ctrl min/max
            sys_ctrl_min = float_or_none(row["Sys Ctrl Min"])
            sys_ctrl_max = float_or_none(row["Sys Ctrl Max"])

            # fan pressures
            f_res_mech = float_or_none(row["FanP Res Mech"])
            f_nonres_intake = float_or_none(row["FanP Nonres Intake"])
            f_nonres_exhaust = float_or_none(row["FanP Nonres Exhaust"])

            # HRV
            hrv_lo = float_or_none(row["HRV Min"])
            hrv_hi = float_or_none(row["HRV Max"])

            # 1) Infiltration
            if inf_min is not None and inf_max is not None:
                if category == "residential":
                    res_infil[bldg_type] = (inf_min, inf_max)
                else:
                    nonres_infil[bldg_type] = (inf_min, inf_max)

            # 2) Year factor range
            # We store for all rows, but typically it's the same for same year_range.
            if yfac_min is not None and yfac_max is not None:
                # If year_range not in year_factor yet, store it:
                # or you can check for consistency
                if year_range not in year_factor:
                    year_factor[year_range] = (yfac_min, yfac_max)
                else:
                    # optionally check if it's the same
                    pass

            # 3) system_type_map => system_letter
            if sys_letter:
                # e.g. system_type_map["residential"]["< 1945"]["Corner House"] = "A"
                if category == "residential":
                    if year_range not in sys_map_res:
                        sys_map_res[year_range] = {}
                    sys_map_res[year_range][bldg_type] = sys_letter
                else:
                    if year_range not in sys_map_nonres:
                        sys_map_nonres[year_range] = {}
                    sys_map_nonres[year_range][bldg_type] = sys_letter

            # 4) system_control_range_[res/nonres]
            # if we have a system letter + sys_ctrl_min/max => we store f_ctrl_range
            if sys_letter and sys_ctrl_min is not None and sys_ctrl_max is not None:
                if category == "residential":
                    if sys_letter not in ctrl_res:
                        ctrl_res[sys_letter] = {"f_ctrl_range": (sys_ctrl_min, sys_ctrl_max)}
                    else:
                        # Could unify minmax or just overwrite
                        pass
                else:
                    if sys_letter not in ctrl_nonres:
                        ctrl_nonres[sys_letter] = {"f_ctrl_range": (sys_ctrl_min, sys_ctrl_max)}
                    else:
                        pass

            # 5) fan_pressure_range
            # We see from example dictionary that there's just a single dict like:
            # fan_pressure_range = {"res_mech": (40, 60), "nonres_intake": (90, 110), ...}
            # If row has "FanP Res Mech" => store in "res_mech"
            if f_res_mech is not None:
                fanp["res_mech"] = (f_res_mech, f_res_mech)
            if f_nonres_intake is not None:
                fanp["nonres_intake"] = (f_nonres_intake, f_nonres_intake)
            if f_nonres_exhaust is not None:
                fanp["nonres_exhaust"] = (f_nonres_exhaust, f_nonres_exhaust)

            # 6) HRV => store min, max
            # We'll unify across rows. Often it's the same in every row, but let's do min-of-min, max-of-max
            if hrv_lo is not None and hrv_hi is not None:
                if hrv_min is None or hrv_max is None:
                    # first time
                    hrv_min, hrv_max = (hrv_lo, hrv_hi)
                else:
                    # unify
                    new_min = min(hrv_min, hrv_lo)
                    new_max = max(hrv_max, hrv_hi)
                    hrv_min, hrv_max = (new_min, new_max)

        # after processing all rows in this group, store final hrv
        subdict["hrv_sensible_eff_range"] = (hrv_min, hrv_max)

    # Write out ventilation_lookup.py
    out_path = os.path.join(os.path.dirname(excel_path), "ventilation_lookup.py")
    with open(out_path, "w", encoding="utf-8") as f:
        f.write("# Auto-generated from Vent_lookup.xlsx\n\n")
        f.write('"""\nventilation_lookup.py\nLarge nested dict from Excel.\n"""\n\n')
        f.write("ventilation_lookup = ")
        f.write(pprint.pformat(ventilation_lookup, sort_dicts=False))
        f.write("\n")

    print(f"Created ventilation_lookup.py at: {out_path}")

if __name__ == "__main__":
    main()


Created ventilation_lookup.py at: D:\Documents\E_Plus_2030_py\lookup_pys\ventilation_lookup.py


In [3]:
import pandas as pd
import os

def clean_numeric(val):
    """
    Convert Excel cell values (possibly '-', NaN, float, etc.)
    into either a float or None.
    """
    if pd.isna(val):
        return None
    if isinstance(val, str) and val.strip() == '-':
        return None
    try:
        return float(val)
    except ValueError:
        return None

def generate_material_data(df, is_residential=True):
    """
    Given a dataframe (either residential or non-residential)
    return a dictionary that matches the desired structure.
    
    For residential:
        key = (building_type, year_range, scenario, calibration_stage)
    For non-residential:
        key = (building_type, year_range, scenario, calibration_stage)
        
    The returned dictionary will look like:
    
    {
      (key_tuple): {
         "roughness": str,
         "wwr_range": (float or None, float or None),
         "material_opaque_lookup": str,
         "material_window_lookup": str,
         "doors": {
             "area_m2": float or None,
             "R_value_range": (float or None, float or None),
             "U_value_range": (float or None, float or None),
             "material_opaque_lookup": str,
             "material_window_lookup": str
         },
         "exterior_wall": { ... },
         ...
      },
      ...
    }
    """
    # Columns we expect:
    # building_function, building_type, year_range, scenario, calibration_stage, element,
    # area_m2, R_value_min, R_value_max, U_value_min, U_value_max,
    # roughness, material_opaque_lookup, material_window_lookup, min_wwr, max_wwr
    #
    # For grouping keys:
    #   residential -> (building_type, year_range, scenario, calibration_stage)
    #   non-residential -> same grouping if you want the first item to be building_type from the column.
    
    # Decide grouping columns based on your logic:
    if is_residential:
        grouping_cols = ["building_type", "year_range", "scenario", "calibration_stage"]
    else:
        # In your examples, "Accommodation Function" was the building_type in the dictionary key.
        # But your sheet might have building_type or building_function swapped.
        # Adjust if needed. For example, if your file uses:
        #   building_function = Non-Residential
        #   building_type = Accommodation Function
        # we group on (building_type, year_range, scenario, calibration_stage)
        grouping_cols = ["building_type", "year_range", "scenario", "calibration_stage"]
    
    # Dictionary to fill
    materials_data = {}
    
    # Group the dataframe by the chosen columns
    grouped = df.groupby(grouping_cols)
    
    for group_key, subdf in grouped:
        # Convert group_key to the tuple we want
        # e.g. ('Apartment', '1946-1964', 'scenario1', 'post_calibration')
        dict_key = tuple(group_key)
        
        # We'll pick the first row in this group for top-level info
        first_row = subdf.iloc[0]
        
        roughness = str(first_row["roughness"]) if not pd.isna(first_row["roughness"]) else ""
        mat_opaque_top = str(first_row["material_opaque_lookup"]) if not pd.isna(first_row["material_opaque_lookup"]) else ""
        mat_window_top = str(first_row["material_window_lookup"]) if not pd.isna(first_row["material_window_lookup"]) else ""
        
        wwr_min = clean_numeric(first_row["min_wwr"])
        wwr_max = clean_numeric(first_row["max_wwr"])
        
        # Start building the dictionary for this group
        materials_data[dict_key] = {
            "roughness": roughness,
            "wwr_range": (wwr_min, wwr_max),
            "material_opaque_lookup": mat_opaque_top,
            "material_window_lookup": mat_window_top
        }
        
        # For each element in subdf, populate sub-dict
        # e.g. doors, windows, exterior_wall, etc.
        for _, row in subdf.iterrows():
            element_name = str(row["element"]).strip()
            
            area_m2 = clean_numeric(row["area_m2"])
            R_value_min = clean_numeric(row["R_value_min"])
            R_value_max = clean_numeric(row["R_value_max"])
            U_value_min = clean_numeric(row["U_value_min"])
            U_value_max = clean_numeric(row["U_value_max"])
            
            mat_opaque = str(row["material_opaque_lookup"]) if not pd.isna(row["material_opaque_lookup"]) else ""
            mat_window = str(row["material_window_lookup"]) if not pd.isna(row["material_window_lookup"]) else ""
            
            # Create sub-dict for the element
            element_dict = {
                "area_m2": area_m2,
                "R_value_range": (R_value_min, R_value_max),
                "U_value_range": (U_value_min, U_value_max),
                "material_opaque_lookup": mat_opaque,
                "material_window_lookup": mat_window
            }
            
            # Insert into the group dictionary
            materials_data[dict_key][element_name] = element_dict
    
    return materials_data

def dict_to_python_file(dictionary_data, variable_name, output_path):
    """
    Write the given dictionary (of dictionaries) to a .py file
    in a nicely formatted way that recreates the structure.
    """
    with open(output_path, 'w', encoding='utf-8') as f:
        f.write(f"{variable_name} = {{\n")
        
        # Sort keys (optional, for consistent output)
        for k in sorted(dictionary_data.keys()):
            f.write(f"    {k}: {{\n")
            
            group_dict = dictionary_data[k]
            
            # We'll separate out the "standard" top-level keys from the elements
            # standard top-level: roughness, wwr_range, material_opaque_lookup, material_window_lookup
            top_keys = ["roughness", "wwr_range", "material_opaque_lookup", "material_window_lookup"]
            elements = {}
            
            # Print top-level items first
            for top_k in top_keys:
                val = group_dict.get(top_k, None)
                if isinstance(val, str):
                    f.write(f'        "{top_k}": "{val}",\n')
                else:
                    f.write(f'        "{top_k}": {val},\n')
            
            # Then print the building elements (doors, windows, etc.)
            # anything else in group_dict that is not in top_keys
            element_keys = [ek for ek in group_dict.keys() if ek not in top_keys]
            for element_key in element_keys:
                elem_data = group_dict[element_key]
                f.write(f'        "{element_key}": {{\n')
                # Now inside each element dict
                for sub_k, sub_v in elem_data.items():
                    if isinstance(sub_v, str):
                        f.write(f'            "{sub_k}": "{sub_v}",\n')
                    else:
                        f.write(f'            "{sub_k}": {sub_v},\n')
                f.write("        },\n")
            
            f.write("    },\n")
        
        f.write("}\n")

def main():
    # Adjust paths as needed
    res_excel = r"D:\Documents\E_Plus_2030_py\lookup_pys\envelop_res5.xlsx"
    nonres_excel = r"D:\Documents\E_Plus_2030_py\lookup_pys\envelop_nonres5.xlsx"
    
    out_residential_py = r"D:\Documents\E_Plus_2030_py\Lookups\data_materials_residential.py"
    out_nonres_py = r"D:\Documents\E_Plus_2030_py\Lookups\data_materials_non_residential.py"
    
    # 1. Read data
    df_res = pd.read_excel(res_excel)
    df_nonres = pd.read_excel(nonres_excel)
    
    # 2. Generate dictionaries
    residential_data = generate_material_data(df_res, is_residential=True)
    non_residential_data = generate_material_data(df_nonres, is_residential=False)
    
    # 3. Write to .py files
    dict_to_python_file(
        dictionary_data=residential_data,
        variable_name="residential_materials_data",
        output_path=out_residential_py
    )
    
    dict_to_python_file(
        dictionary_data=non_residential_data,
        variable_name="non_residential_materials_data",
        output_path=out_nonres_py
    )

if __name__ == "__main__":
    main()
